<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

## Librerías

In [3]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
import json
import os
import io
import ast
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import sweetviz as sv
import webbrowser

<h2> Google Cloud Platform </h2>

In [4]:
# Google Cloud Platform
# Obtener el proyecto actual
project_id = os.environ.get("Proyecto Final - Henry")

# Crear un cliente de Cloud Storage
client = storage.Client(project=project_id)

# Obtener el nombre del bucket
bucket_name = "gmy"

# Obtener el objeto de referencia al bucket
bucket = client.bucket(bucket_name)

In [5]:
# Obtener el blob
blob = bucket.get_blob('etl/metadata-sitios-category.parquet')

# Descargar el blob
data = blob.download_as_string()

# Convertir el objeto de bytes a un objeto similar a un archivo
file = io.BytesIO(data)

# Leer el archivo parquet en un DataFrame
df = pd.read_parquet(file)

# Imprimir el DataFrame
df

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,[Pharmacy]
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,[Textile exporter]
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant]
3,0x80c2c89923b27a41:0x32041559418d447,[Fabric store]
4,0x80c2c632f933b073:0xc31785961fe826a6,[Fabric store]
...,...,...
3025006,0x89c6c74f43a49b55:0x6be6995921c58b12,"[Steak house, Fine dining restaurant, Seafood ..."
3025007,0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,[Chevrolet dealer]
3025008,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,[Veterinarian]
3025009,0x88d9e872a9e39f3d:0x5de29f50507e5d73,[Animal feed store]


In [11]:
df = df.dropna(subset=["category"])

In [12]:
df

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,[Pharmacy]
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,[Textile exporter]
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant]
3,0x80c2c89923b27a41:0x32041559418d447,[Fabric store]
4,0x80c2c632f933b073:0xc31785961fe826a6,[Fabric store]
...,...,...
3025006,0x89c6c74f43a49b55:0x6be6995921c58b12,"[Steak house, Fine dining restaurant, Seafood ..."
3025007,0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,[Chevrolet dealer]
3025008,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,[Veterinarian]
3025009,0x88d9e872a9e39f3d:0x5de29f50507e5d73,[Animal feed store]


In [24]:
df.iloc[1502]["category"]

array(['Discount supermarket', 'Discount store', 'Dollar store'],
      dtype=object)

In [25]:
category_dict_str = []
row = df.iloc[1502]
category_list = row["category"]
if category_list is not None:
    for cat in category_list:
        category_dict_str.append({
            "gmapid": row["gmap_id"],
            "categoria": cat                
        })
        print(row["gmap_id"],cat)
category_dict_str

0x80dcc9b1b0517d65:0x70bc4c6d139812d6 Discount supermarket
0x80dcc9b1b0517d65:0x70bc4c6d139812d6 Discount store
0x80dcc9b1b0517d65:0x70bc4c6d139812d6 Dollar store


[{'gmapid': '0x80dcc9b1b0517d65:0x70bc4c6d139812d6',
  'categoria': 'Discount supermarket'},
 {'gmapid': '0x80dcc9b1b0517d65:0x70bc4c6d139812d6',
  'categoria': 'Discount store'},
 {'gmapid': '0x80dcc9b1b0517d65:0x70bc4c6d139812d6',
  'categoria': 'Dollar store'}]

In [26]:
# Crear una lista de diccionarios con los datos del DataFrame
category_dict_str = []
for i in range(len(df)):
    row = df.iloc[i]
    category_list = row["category"]
    if category_list is not None:
        for cat in category_list:
            category_dict_str.append({
                "gmapid": row["gmap_id"],
                "categoria": cat                
            })

# Crear un DataFrame a partir de la lista de diccionarios
df_categoria = pd.DataFrame(category_dict_str)

# Eliminar duplicados
df_categoria = df_categoria.drop_duplicates(["gmapid", "categoria"])

# Guardar el DataFrame en un CSV
df_categoria.to_csv(bucket.blob("etl/metadata-sitios-category-detalle.csv").open("w"), index=False)
# Guardar el DataFrame en un parquet
df_categoria.to_parquet(bucket.blob("etl/metadata-sitios-category-detalle.parquet").open("wb"))
 

In [27]:
df_categoria

,gmapid,categoria
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,Textile exporter
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
3,0x80c2c89923b27a41:0x32041559418d447,Fabric store
4,0x80c2c632f933b073:0xc31785961fe826a6,Fabric store
...,...,...
6262691,0x89c6c74f43a49b55:0x6be6995921c58b12,Seafood restaurant
6262692,0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,Chevrolet dealer
6262693,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,Veterinarian
6262694,0x88d9e872a9e39f3d:0x5de29f50507e5d73,Animal feed store


In [28]:
df_categoria.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6211324 entries, 0 to 6262695
Data columns (total 2 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   gmapid     object
 1   categoria  object
dtypes: object(2)
memory usage: 142.2+ MB


In [29]:
df_categoria.describe()

,gmapid,categoria
count,6211324,6211324
unique,2981245,4471
top,0x89c24484b823fed3:0x74f6329f85c4f381,Restaurant
freq,32,96738


In [30]:
# Generar el informe de análisis
profile = pandas_profiling.ProfileReport(df_categoria)

# Mostrar el informe
profile.to_file("metadata-sitios-category-detalle.html")
webbrowser.open("metadata-sitios-category-detalle.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

False